# KLEIN Implementation

- Three versions exist based on key size:
    - KLEIN-64 (12 Rounds)
    - KLEIN-80 (16 Rounds)
    - KLEIN-96 (20 Rounds)

- Following is the pseudocode  
```
    sk1 ← KEY;  
    STATE ← PLAINTEXT;  
    
    for i = 1 to NR do  
        AddRoundKey(STATE, ski);  
        SubNibbles(STATE);  
        RotateNibbles(STATE);  
        MixNibbles(STATE);  
        ski+1 ← KeySchedule(ski, i);  
    end for  
    
    CIPHERTEXT ← AddRoundKey(STATE, skNR+1);  
```

- The following functionalities need to be implemented:
    - Add Round Key
    - SubNibbles
    - MixNibbles
    - KeySchedule

In [1]:
s_box = {   0x0:0x7, 0x1:0x4, 0x2:0xA, 0x3:0x9, 0x4:0x1, 0x5:0xF, 0x6:0xB, 0x7:0x0,
            0x8:0xC, 0x9:0x3, 0xA:0x2, 0xB:0x6, 0xC:0x8, 0xD:0xE, 0xE:0xD, 0xF:0x5   }

def SubNibbles(state, reverse=False):
    return [s_box[byte] for byte in state]

def RotateNibbles(state, reverse=False):
    if not reverse:
        return state[4:]+state[:4]
    else:
        return state[-4:]+state[:-4]
    
def AddRoundKey(state, subkey):
    for index in range(len(state)):
        state[index] ^= subkey[index]
    return state

def gf_mul(a, b):
    """Multiply two 8-bit integers in GF(2^8) modulo x^8 + x^4 + x^3 + x + 1 (0x11B)."""
    p = 0  
    for _ in range(8):  
        if b & 1: 
            p ^= a  
        b >>= 1  
        a <<= 1  
        if a & 0x100:  # If the shifted-out bit is 1 (overflow for GF(2^8))
            a ^= 0x11B  # Reduce modulo the irreducible polynomial (0x11B)
    return p & 0xFF  # Ensure the result is an 8-bit value

def MixNibbles(state, reverse=False):
    """Perform the MixNibbles transformation on the state."""
    # MixNibbles matrix depending on whether reverse transformation is needed
    if not reverse:
        mix_matrix = [
            [2, 3, 1, 1],
            [1, 2, 3, 1],
            [1, 1, 2, 3],
            [3, 1, 1, 2]
        ]
    else:
        mix_matrix = [
            [0x0E, 0x0B, 0x0D, 0x09],
            [0x09, 0x0E, 0x0B, 0x0D],
            [0x0D, 0x09, 0x0E, 0x0B],
            [0x0B, 0x0D, 0x09, 0x0E]
        ]

    def process_half(half):
        """Process a half of the state with the MixNibbles matrix."""
        # Form the 4x1 input matrix by grouping nibbles into bytes
        input_matrix = [
            (half[0] << 4) | half[1],
            (half[2] << 4) | half[3],
            (half[4] << 4) | half[5],
            (half[6] << 4) | half[7]
        ]

        
        output_matrix = [0] * 4

        # Multiply the mix_matrix with the input_matrix
        for row in range(4):
            for col in range(4):
                output_matrix[row] ^= gf_mul(mix_matrix[row][col], input_matrix[col])

        output = []
        for value in output_matrix:
            output.append((value >> 4) & 0xF)  
            output.append(value & 0xF) 
        return output

    half1, half2 = state[:8], state[8:]
    processed_half1 = process_half(half1)
    processed_half2 = process_half(half2)
    return processed_half1 + processed_half2



def KeySchedule(subkey, round):
    sk_length = len(subkey)
    left_half, right_half = subkey[:sk_length//2], subkey[sk_length//2:]
    left_half = left_half[2:]+left_half[:2]
    right_half = right_half[2:]+right_half[:2]

    new_sk_left = right_half[:]
    new_sk_left[4] ^= round >> 4
    new_sk_left[5] ^= round & 0xF

    new_sk_right = []
    for i,j in zip (left_half,right_half):
        new_sk_right.append(i^j)
    for k in range(2,6):
        new_sk_right[k] = s_box[new_sk_right[k]]

    return new_sk_left + new_sk_right

In [2]:
init_key = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
prev_key = init_key
for i in range(0,12):
    prev_key = KeySchedule(prev_key,i)
    print(prev_key)

[10, 11, 12, 13, 14, 15, 8, 9, 8, 8, 12, 12, 12, 12, 8, 8]
[12, 12, 12, 12, 8, 9, 8, 8, 0, 1, 10, 9, 7, 4, 2, 3]
[10, 9, 7, 4, 2, 1, 0, 1, 6, 5, 5, 14, 2, 6, 12, 13]
[5, 14, 2, 6, 12, 14, 6, 5, 2, 10, 7, 0, 8, 8, 12, 12]
[7, 0, 8, 8, 12, 8, 2, 10, 5, 6, 1, 11, 2, 3, 7, 4]
[1, 11, 2, 3, 7, 1, 5, 6, 9, 3, 13, 6, 15, 13, 2, 6]
[13, 6, 15, 13, 2, 0, 9, 3, 15, 5, 12, 8, 0, 7, 8, 8]
[12, 8, 0, 7, 8, 15, 15, 5, 3, 5, 10, 0, 4, 6, 2, 3]
[10, 0, 4, 6, 2, 11, 3, 5, 10, 7, 8, 3, 14, 11, 15, 13]
[8, 3, 14, 11, 15, 4, 10, 7, 12, 5, 8, 7, 8, 12, 0, 7]
[8, 7, 8, 12, 0, 13, 12, 5, 6, 12, 0, 12, 2, 7, 4, 6]
[0, 12, 2, 7, 4, 13, 6, 12, 8, 0, 10, 2, 12, 9, 14, 11]


In [5]:
def inverseKeySchedule(subkey, round):
    sk_length = len(subkey)

    prev_sk_right = subkey[:sk_length//2]
    prev_sk_right[4] ^= round >> 4
    prev_sk_right[5] ^= round & 0xF

    sk_left = subkey[sk_length//2:]
    prev_sk_left = []
    for k in range(2,6):
        sk_left[k] = s_box[sk_left[k]]
    for i,j in zip (sk_left,prev_sk_right):
        prev_sk_left.append(i^j)

    prev_sk_left = prev_sk_left[-2:] + prev_sk_left[:-2]
    prev_sk_right = prev_sk_right[-2:] + prev_sk_right[:-2]

    return prev_sk_left + prev_sk_right

In [6]:
init_key = [0, 12, 2, 7, 4, 13, 6, 12, 8, 0, 10, 2, 12, 9, 14, 11]
prev_key = init_key
for i in range(11,-1,-1):
    prev_key = inverseKeySchedule(prev_key,i)
    print(prev_key)

[8, 7, 8, 12, 0, 13, 12, 5, 6, 12, 0, 12, 2, 7, 4, 6]
[8, 3, 14, 11, 15, 4, 10, 7, 12, 5, 8, 7, 8, 12, 0, 7]
[10, 0, 4, 6, 2, 11, 3, 5, 10, 7, 8, 3, 14, 11, 15, 13]
[12, 8, 0, 7, 8, 15, 15, 5, 3, 5, 10, 0, 4, 6, 2, 3]
[13, 6, 15, 13, 2, 0, 9, 3, 15, 5, 12, 8, 0, 7, 8, 8]
[1, 11, 2, 3, 7, 1, 5, 6, 9, 3, 13, 6, 15, 13, 2, 6]
[7, 0, 8, 8, 12, 8, 2, 10, 5, 6, 1, 11, 2, 3, 7, 4]
[5, 14, 2, 6, 12, 14, 6, 5, 2, 10, 7, 0, 8, 8, 12, 12]
[10, 9, 7, 4, 2, 1, 0, 1, 6, 5, 5, 14, 2, 6, 12, 13]
[12, 12, 12, 12, 8, 9, 8, 8, 0, 1, 10, 9, 7, 4, 2, 3]
[10, 11, 12, 13, 14, 15, 8, 9, 8, 8, 12, 12, 12, 12, 8, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


### Now that all functions have been implemented, let's move on to:
- Encryption
- Decryption

In [2]:
def encrypt_klein(message, masterkey):
    """Encrypts message using masterkey. 
    Message: List of 4-bit integers (Allowed length: 16)
    Masterkey: List of 4-bit integers (Allowed lengths: 16, 20, 24)"""
    state = message
    subkey = masterkey

    rounds_conversion = { 16:12, 20:16, 24:20 }
    rounds = rounds_conversion[len(masterkey)]
    for i in range(1,rounds+1):
        state = AddRoundKey(state, subkey)
        state = SubNibbles(state)
        state = RotateNibbles(state)
        state = MixNibbles(state)
        subkey = KeySchedule(subkey, i)

    ciphertext = AddRoundKey(state, subkey)
    return ciphertext

def decrypt_klein(ciphertext, masterkey):
    """Decrypts ciphertext using masterkey. 
    Ciphertext: List of 4-bit integers (Allowed length: 16)
    Masterkey: List of 4-bit integers (Allowed lengths: 16, 20, 24)"""
    subkey = masterkey
    
    rounds_conversion = { 16:12, 20:16, 24:20 }
    rounds = rounds_conversion[len(masterkey)]
    keys = [subkey]
    for i in range(1,rounds+1):
        keys.append(KeySchedule(keys[-1],i))
    keys.reverse()
    state = AddRoundKey(ciphertext, keys[0])
    for i in range(1,rounds+1):
        state = MixNibbles(state, reverse=True)
        state = RotateNibbles(state, reverse=True)
        state = SubNibbles(state, reverse=True)
        state = AddRoundKey(state, keys[i])
    
    return state

### Testing Encryption and Decryption

In [3]:
def hex_to_nibbles(hex_str):
    """Convert a hexadecimal string to an array of 4-bit integers (nibbles)."""
    nibbles = []
    for char in hex_str:
        nibble = int(char, 16)
        nibbles.append(nibble)
    return nibbles

def nibbles_to_hex(nibbles):
    """Convert an array of 4-bit integers (nibbles) to a hexadecimal string."""
    return ''.join([hex(x)[2:] for x in nibbles])

# Test vectors from the Appendix-A of Research Paper pdf
test_vectors = {
    "KLEIN-64": [
        {"key": "0000000000000000", "message": "FFFFFFFFFFFFFFFF", "cipher": "CDC0B51F14722BBE"},
        {"key": "FFFFFFFFFFFFFFFF", "message": "0000000000000000", "cipher": "6456764E8602E154"},
        {"key": "1234567890ABCDEF", "message": "FFFFFFFFFFFFFFFF", "cipher": "592356C4997176C8"},
        {"key": "0000000000000000", "message": "1234567890ABCDEF", "cipher": "629F9D6DFF95800E"}
    ],
    "KLEIN-80": [
        {"key": "00000000000000000000", "message": "FFFFFFFFFFFFFFFF", "cipher": "6677E20D1A53A431"},
        {"key": "FFFFFFFFFFFFFFFFFFFF", "message": "0000000000000000", "cipher": "82247502273DCC5F"},
        {"key": "1234567890ABCDEF1234", "message": "FFFFFFFFFFFFFFFF", "cipher": "3F210F67CB23687A"},
        {"key": "00000000000000000000", "message": "1234567890ABCDEF", "cipher": "BA5239E93E784366"}
    ],
    "KLEIN-96": [
        {"key": "000000000000000000000000", "message": "FFFFFFFFFFFFFFFF", "cipher": "DB9FA7D33D8E8E36"},
        {"key": "FFFFFFFFFFFFFFFFFFFFFFFF", "message": "0000000000000000", "cipher": "15A3A03386A7FEC6"},
        {"key": "1234567890ABCDEF12345678", "message": "FFFFFFFFFFFFFFFF", "cipher": "79687798AFDA0BC3"},
        {"key": "000000000000000000000000", "message": "1234567890ABCDEF", "cipher": "5006A987A500BFDD"}
    ]
}


def test_encrypt_klein():
    for variant, vectors in test_vectors.items():
        print(f"Testing {variant}")
        for i, vector in enumerate(vectors):
            key = hex_to_nibbles(vector["key"])
            message = hex_to_nibbles(vector["message"])
            expected_cipher = vector["cipher"]
            
            # Encrypt the message, convert result to hex, then decrypt to get original message back
            result = encrypt_klein(message, key)
            result_hex = nibbles_to_hex(result)
            decrypt_result = decrypt_klein(hex_to_nibbles(result_hex),key)
            decrypt_hex = nibbles_to_hex(decrypt_result)
            
            # Compare obtained cipher with expected cipher, and also compare decrypted message is equal to original message
            # Print 'PASS' only if both encryption and decryption are working correctly
            if result_hex.upper() == expected_cipher and decrypt_hex.upper() == vector['message']:
                print(f"  Test {i + 1}: PASS")
            else:
                print(f"  Test {i + 1}: FAIL (Expected: {expected_cipher}, Got: {result_hex})")

test_encrypt_klein()


Testing KLEIN-64
  Test 1: PASS
  Test 2: PASS
  Test 3: PASS
  Test 4: PASS
Testing KLEIN-80
  Test 1: PASS
  Test 2: PASS
  Test 3: PASS
  Test 4: PASS
Testing KLEIN-96
  Test 1: PASS
  Test 2: PASS
  Test 3: PASS
  Test 4: PASS


### We have successfully implemented encryption and decryption of fixed block length (64 bits in this case).
### Now we need to implement some mode of operation to use this on arbitrary length string.

### Implementing CBC mode of operation - Cipher Block Chaining

In [4]:
def xor_nibbles(block1, block2):
    """Perform XOR on two blocks of nibbles (4-bit integers)."""
    return [x ^ y for x, y in zip(block1, block2)]

def string_to_nibbles(string):
    """Convert a string to a list of 4-bit integers (nibbles) using ASCII values."""
    nibbles = []
    for char in string:
        ascii_val = ord(char)
        nibbles.append((ascii_val >> 4) & 0xF)  # High nibble
        nibbles.append(ascii_val & 0xF)         # Low nibble
    return nibbles

def nibbles_to_string(nibbles):
    """Convert a list of 4-bit integers (nibbles) back to a string."""
    chars = []
    for i in range(0, len(nibbles), 2):
        char = (nibbles[i] << 4) | nibbles[i + 1]  # Combine two nibbles into a byte
        chars.append(chr(char))
    return ''.join(chars)

def pad_nibbles(nibbles, block_size=16):
    """
    Pad a list of nibbles using ANSI X.923 padding.
    - nibbles: List of 4-bit integers to be padded.
    - block_size: The block size to pad to (in nibbles).
    """
    padding_length = block_size - (len(nibbles) % block_size)
    padding = [0] * (padding_length - 1) + [padding_length-1]
    return nibbles + padding

def unpad_nibbles(nibbles):
    """
    Remove ANSI X.923 padding from a list of nibbles.
    - nibbles: List of 4-bit integers with padding.
    """
    padding_length = nibbles[-1]+1
    return nibbles[:-padding_length]

def encrypt_cbc(plaintext, masterkey, iv):
    """
    Encrypt a plaintext string using CBC mode.
    - plaintext: The input string to encrypt.
    - masterkey: List of 4-bit integers for the key.
    - iv: Initialization vector as a list of 4-bit integers (length = 16).
    """
    plaintext_nibbles = string_to_nibbles(plaintext)
    padded_plaintext = pad_nibbles(plaintext_nibbles)

    ciphertext = []
    previous_block = iv

    for i in range(0, len(padded_plaintext), 16):
        block = padded_plaintext[i:i + 16]
        xor_block = xor_nibbles(block, previous_block)
        encrypted_block = encrypt_klein(xor_block, masterkey)
        ciphertext.extend(encrypted_block)
        previous_block = encrypted_block  # Update the chaining block

    return nibbles_to_hex(ciphertext)

def decrypt_cbc(ciphertext, masterkey, iv):
    """
    Decrypt a ciphertext list using CBC mode.
    - ciphertext: The encrypted list of 4-bit integers.
    - masterkey: List of 4-bit integers for the key.
    - iv: Initialization vector as a list of 4-bit integers (length = 16).
    """
    plaintext = []
    previous_block = iv
    ciphertext = hex_to_nibbles(ciphertext)

    for i in range(0, len(ciphertext), 16):
        block = ciphertext[i:i + 16]
        copy_block = block[:]
        decrypted_block = decrypt_klein(block, masterkey)
        plaintext_block = xor_nibbles(decrypted_block, previous_block)
        plaintext.extend(plaintext_block)
        previous_block = copy_block  # Update the chaining block

    unpadded_plaintext = unpad_nibbles(plaintext)
    return nibbles_to_string(unpadded_plaintext)


### Let's Try Out Whether CBC mode is working fine or not.

In [ ]:
import secrets

plaintext = "Is KLEIN CBC working????"
masterkey = [secrets.randbelow(16) for _ in range(16)]  # 16 nibbles - random - can be 16/20/24 nibbles for KLEIN-64/80/96
iv = [secrets.randbelow(16) for _ in range(16)]      # 16 nibbles - random

print(f'Original Plaintext: {plaintext}')

ciphertext = encrypt_cbc(plaintext, masterkey, iv)
print(f"Ciphertext: {ciphertext}")

decrypted_plaintext = decrypt_cbc(ciphertext, masterkey, iv)
print(f"Decrypted Plaintext: {decrypted_plaintext}")

Original Plaintext: Is KLEIN CBC working????
Ciphertext: a539606af2bcee2b87ff51b0f2d06d58a3087113a339cf41a9be2280aa11bc5f
Decrypted Plaintext: Is KLEIN CBC working????


### Now we may encrypt any length message string using KLEIN-64/80/96

In [13]:
plaintext = "Is KLEIN CBC working or not????"
mk = '1234567890abcdef'
masterkey = [int(x,16) for x  in mk]
iv = [secrets.randbelow(16) for _ in range(16)]      # 16 nibbles - random

print(f'Original Plaintext: {plaintext}')

ciphertext = encrypt_cbc(plaintext, masterkey, iv)
print(f"Ciphertext: {ciphertext}")

decrypted_plaintext = decrypt_cbc(ciphertext, masterkey, iv)
print(f"Decrypted Plaintext: {decrypted_plaintext}")

Original Plaintext: Is KLEIN CBC working or not????
Ciphertext: 867ae60dd7117db99340997273e91d6fa6437de168af92a2a693b8eb760224c3
Decrypted Plaintext: Is KLEIN CBC working or not????
